<a href="https://colab.research.google.com/github/pgurazada/fast-fast-ai/blob/master/keras_functional_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Replicated from 

- https://machinelearningmastery.com/keras-functional-api-deep-learning/
- https://medium.com/tensorflow/predicting-the-price-of-wine-with-the-keras-functional-api-and-tensorflow-a95d1c2c1b03

In [0]:
from keras.utils import plot_model

In [0]:
from keras.models import Model

from keras.layers import Input, Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import TimeDistributed

from keras.layers import Concatenate

# Multilayer Perceptron

In [0]:
input_layer = Input(shape=(10, ))

hidden1 = Dense(10, activation='relu')(input_layer)
hidden2 = Dense(20, activation='relu')(hidden1)
hidden3 = Dense(10, activation='relu')(hidden2)

output = Dense(1, activation='sigmoid')(hidden3)

model = Model(inputs=input_layer, outputs=output)

In [0]:
model.summary()

In [0]:
plot_model(model, to_file='mlp-model.png')

# Convolutional Neural Network

In [0]:
input_layer = Input(shape=(64,64,1))

conv1 = Conv2D(32, kernel_size=4, activation='relu')(input_layer)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(16, kernel_size=4, activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

flat = Flatten()(pool2)

hidden1 = Dense(10, activation='relu')(flat)

output = Dense(1, activation='sigmoid')(hidden1)

model = Model(inputs=input_layer, outputs=output)

In [0]:
model.summary()

In [0]:
plot_model(model, to_file='cnn-model.png')

# Recurrent Neural Network

In [0]:
input_layer = Input(shape=(100,1))

hidden1 = LSTM(10)(input_layer)
hidden2 = Dense(10, activation='relu')(hidden1)

output = Dense(1, activation='sigmoid')(hidden2)

model = Model(inputs=input_layer, outputs=output)

In [0]:
model.summary()

In [0]:
plot_model(model, to_file='rnn-model.png')

# Shared input model

In [0]:
# input layer
input_layer = Input(shape=(64,64,1))

# first feature extractor
conv1 = Conv2D(32, kernel_size=4, activation='relu')(input_layer)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
flat1 = Flatten()(pool1)

# second feature extractor
conv2 = Conv2D(16, kernel_size=8, activation='relu')(input_layer)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flat2 = Flatten()(pool2)

# merge feature extractors
merge = Concatenate()([flat1, flat2])

# interpretation layer
hidden1 = Dense(10, activation='relu')(merge)

# prediction output
output = Dense(1, activation='sigmoid')(hidden1)

model = Model(inputs=input_layer, outputs=output)

In [0]:
model.summary()

In [0]:
plot_model(model, 'shared-input-model.png')

# Shared feature extraction model

In [0]:
# define input
input_layer = Input(shape=(100,1))

# feature extraction
extract1 = LSTM(10)(input_layer)

# first interpretation model
interp1 = Dense(10, activation='relu')(extract1)

# second interpretation model
interp11 = Dense(10, activation='relu')(extract1)
interp12 = Dense(20, activation='relu')(interp11)
interp13 = Dense(10, activation='relu')(interp12)

# merge interpretation
merge = Concatenate()([interp1, interp13])

# output
output = Dense(1, activation='sigmoid')(merge)

model = Model(inputs=input_layer, outputs=output)

In [0]:
model.summary()

In [0]:
plot_model(model, to_file='shared-feature-extractors.png')

# Multiple Inputs Model

In [0]:
# first input model
input1 = Input(shape=(64, 64, 1)) # only one color channel
conv11 = Conv2D(32, kernel_size=4, activation='relu')(input1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)

# second input model
input2 = Input(shape=(32, 32, 3)) # three color channels
conv21 = Conv2D(32, kernel_size=4, activation='relu')(input2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)

# merge input models
merge = Concatenate()([flat1, flat2])

# interpretation model
hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(10, activation='relu')(hidden1)

output = Dense(1, activation='sigmoid')(hidden2)

model = Model(inputs=[input1, input2], outputs=output)

In [0]:
model.summary()

In [0]:
plot_model(model, to_file='multi-input-model.png')

# Multiple outputs model

In [0]:
# input layer
input_layer = Input(shape=(100,1))

# feature extraction
extract = LSTM(10, return_sequences=True)(input_layer)

# classification output
class11 = LSTM(10)(extract)
class12 = Dense(10, activation='relu')(class11)
output1 = Dense(1, activation='sigmoid')(class12)

# sequence output
output2 = TimeDistributed(Dense(1, activation='linear'))(extract)

# output
model = Model(inputs=input_layer, outputs=[output1, output2])

In [0]:
model.summary()

In [0]:
plot_model(model, to_file='multi-output-model.png')

Best practises:

- Consistent Variable Names. Use the same variable name for the input (visible) and output layers (output) and perhaps even the hidden layers (hidden1, hidden2). It will help to connect things together correctly.
- Review Layer Summary. Always print the model summary and review the layer outputs to ensure that the model was connected together as you expected.
- Review Graph Plots. Always create a plot of the model graph and review it to ensure that everything was put together as you intended.
- Name the layers. You can assign names to layers that are used when reviewing summaries and plots of the model graph. For example: Dense(1, name=’hidden1′).
- Separate Submodels. Consider separating out the development of submodels and combine the submodels together at the end.

# Worked out example

In [0]:
!wget -q https://storage.googleapis.com/sara-cloud-ml/wine_data.csv

In [0]:
import os
import numpy as np
import pandas as pd

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [0]:
data_df = pd.read_csv("wine_data.csv").dropna()

In [0]:
data_df.isnull().any()

In [0]:
data_df.head()

In [0]:
train_size = int(len(data_df) * .8)

# Train features
description_train = data_df['description'][:train_size]
variety_train = data_df['variety'][:train_size]

# Train labels
labels_train = data_df['price'][:train_size]

In [0]:
vocab_size = 12000
tokenize = Tokenizer(num_words=vocab_size, char_level=False)

In [0]:
tokenize.fit_on_texts(description_train) # only fit on train

In [0]:
encoder = LabelEncoder()
variety_train = encoder.fit_transform(variety_train)

In [0]:
num_classes = np.max(variety_train) + 1

In [0]:
variety_train = to_categorical(variety_train, num_classes)

In [0]:
variety_train.shape

In [0]:
description_bow_train = tokenize.texts_to_matrix(description_train)

In [0]:
description_bow_train.shape

## 1. Wide model

In [0]:
bow_inputs = Input(shape=(vocab_size,))
variety_inputs = Input(shape=(num_classes,))

merged_layer = Concatenate()([bow_inputs, variety_inputs])

hidden1 = Dense(256, activation='relu')(merged_layer)
output = Dense(1)(hidden1)

wide_model = Model(inputs=[bow_inputs, variety_inputs], outputs=output)

In [0]:
wide_model.summary()

In [0]:
wide_model.compile(loss='mse', 
                   optimizer='adam', 
                   metrics=['accuracy'])

## 2. Deep Model

In [0]:
train_embed = tokenize.texts_to_sequences(description_train)

In [0]:
max_seq_length = 170
train_embed = pad_sequences(train_embed, maxlen=max_seq_length)

In [0]:
deep_inputs = Input(shape=(max_seq_length,))

embedding = Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = Flatten()(embedding)


embed_out = Dense(1, activation='linear')(embedding)

deep_model = Model(inputs=deep_inputs, outputs=embed_out)

In [0]:
deep_model.summary()

In [0]:
deep_model.compile(loss='mse', 
                   optimizer='adam', 
                   metrics=['accuracy'])

## 3. Putting the deep and wide model together

In [0]:
merged_out = Concatenate()([wide_model.output, deep_model.output])

final_output = Dense(1)(merged_out)

combined_model = Model(wide_model.input + [deep_model.input], final_output)

In [0]:
combined_model.summary()

In [0]:
plot_model(combined_model, to_file='wide-deep-wine-model.png')

In [0]:
combined_model.compile(loss='mse',
                       optimizer='adam')

In [0]:
combined_model.fit([description_bow_train, variety_train] + [train_embed], 
                   labels_train, 
                   epochs=2, 
                   batch_size=128)